In [10]:
import time
from pinecone import Pinecone, ServerlessSpec
from langchain_groq import ChatGroq


In [11]:
# Initialize Pinecone
def initialize_pinecone(api_key, environment):
    return Pinecone(api_key=api_key, environment=environment)


In [12]:

# Create or connect to a Pinecone index
def create_or_connect_index(pc, index_name, dimension, metric, cloud, region):
    if index_name not in pc.list_indexes().names():
        pc.create_index(
            name=index_name,
            dimension=dimension,
            metric=metric,
            spec=ServerlessSpec(cloud=cloud, region=region)
        )
        print(f"Index '{index_name}' created.")
    else:
        print(f"Index '{index_name}' already exists.")
    return pc.Index(index_name)

# Wait for the index to be ready
def wait_for_index_ready(pc, index_name):
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)
    print(f"Index '{index_name}' is ready.")



In [13]:
# Upsert records into the Pinecone index
def upsert_records(index, namespace, data, embeddings):
    records = []
    for d, e in zip(data, embeddings):
        records.append({
            "id": d["question"],
            "values": e["values"],
            "metadata": {"text": d["question_paragraph"]}
        })
    index.upsert(vectors=records, namespace=namespace)
    print(f"Upserted {len(records)} records into namespace '{namespace}'.")


In [14]:
# Query Pinecone for relevant contexts
def query_pinecone(index, namespace, query_embedding, top_k=6):
    results = index.query(
        namespace=namespace,
        vector=query_embedding[0].values,
        top_k=top_k,
        include_metadata=True
    )
    return results


In [15]:
# Generate embeddings using Pinecone inference
def generate_embeddings(pc, model, inputs, input_type):
    return pc.inference.embed(
        model=model,
        inputs=inputs,
        parameters={"input_type": input_type, "truncate": "END"}
    )

In [16]:
# Use ChatGroq to generate a quiz
def generate_quiz_with_groq(llm, retrieved_context):
    groq_prompt = f"""
    Based on the following context, create a quiz with 10 multiple-choice questions:
    {retrieved_context}
    """
    response = llm.invoke(groq_prompt)
    return response.content


In [17]:
# Main workflow
def main():
    # Initialize Pinecone
    pc = initialize_pinecone(
        api_key="pcsk_3CYnJi_TZbGr8CeCcVxAsz4Li7J5n5hNBRqM7PA7k6xGKx7ftNXUYMYUJLJcb3PZrTneH4",
        environment="us-west1-gcp"
    )


    # Use ChatGroq to generate the quiz
    llm = ChatGroq(
        temperature=0,
        groq_api_key="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",
        model_name="llama-3.1-70b-versatile"
    )
    # Define parameters
    index_name = "quiz-index"
    namespace = "example-namespace"
    model_name = "multilingual-e5-large"
    data = [
        {"question": "Q1", "question_paragraph": "What is regression?"},
        {"question": "Q2", "question_paragraph": "Explain linear regression."}
    ]
    query = "Explain regression."

    # Create or connect to the index
    #index = create_or_connect_index(
    #    pc, index_name, dimension=1024, metric="cosine", cloud="aws", region="us-east-1"
    #)

    # Wait for the index to be ready
    #wait_for_index_ready(pc, index_name)

    # Generate embeddings for the data
    #embeddings = generate_embeddings(
    #    pc, model=model_name,
    #    inputs=[d["question_paragraph"] for d in data],
    #    input_type="passage"
    #)

    # Upsert records into the index
    #upsert_records(index, namespace, data, embeddings)

    # Generate embedding for the query
    #query_embedding = generate_embeddings(pc, model=model_name, inputs=[query], input_type="query")

    # Query the Pinecone index
    #results = query_pinecone(index, namespace, query_embedding)
    # Retrieve the context
    #retrieved_context = "\n".join([match["metadata"]["text"] for match in results["matches"]])
    #print("Retrieved Context:", retrieved_context)

    
    #quiz = generate_quiz_with_groq(llm, retrieved_context)
    #print("Generated Quiz:", quiz)
    def delete_all_indexes(pc):
        all_indexes = pc.list_indexes().names()
        for index_name in all_indexes:
            pc.delete_index(index_name)
            print(f"Deleted index: {index_name}")
        print("All indexes deleted.")
    delete_all_indexes(pc)
    # Optional: Delete all indexes after testing
    # delete_all_indexes(pc)

if __name__ == "__main__":
    main()


Deleted index: mcq-index
All indexes deleted.
